In [2]:
import ast
import os
import csv
import datetime 
from datetime import datetime
import fiona
import requests as r
import numpy as np
import pandas as pd
import geopandas as gp
import glob
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
import json
from rasterio.mask import mask
from rasterio.enums import Resampling
from rasterio.shutil import copy
from pathlib import Path
import pickle
import pyproj
from pyproj import Proj
from shapely.ops import transform
import xarray as xr
import geoviews as gv
from cartopy import crs
import hvplot.xarray
import holoviews as hv
gv.extension('bokeh', 'matplotlib')

# Set Up Working Environment
inDir = os.getcwd()
os.chdir(inDir)

In [ ]:
##########################################################################
# clip the Illinois shape from raw Daymet data

mydir = '.../Illisoy/WEATHER/DaymetRaw/daymet_vp'   # firstly 1980-2022 data should be downloaded there

for year in [str(x) for x in list(range(1980, 2023))]:

    sPlist = []

    sPlist += [each for each in os.listdir(mydir) if each.endswith( year + '.nc')]

    directory='.../Illisoy/Illinois_shp/illin.shp'

    for filename in sPlist:

        with fiona.open(directory, "r") as shapefile:
            shapes = [feature["geometry"] for feature in shapefile]

        with rio.open(os.path.join(mydir, filename)) as src:
            out_image, out_transform = rio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        dirname = '.../Illisoy/WEATHER/Illinois/CLIP/Illi19802022/'

        with rio.open(os.path.join(dirname + year + 'vp' + '.nc'), "w", **out_meta) as dest:
            dest.write(out_image)
    ##########################################################################

In [1]:
mydir = '.../Illisoy/WEATHER/Illinois/CLIP/Illi19802022'

sPlist = []

sPlist += [each for each in os.listdir(mydir) if each.endswith('.nc')]

sPlist

NameError: name 'os' is not defined

In [ ]:
### before clip the county polygons, reprojection

from rasterio.warp import calculate_default_transform, reproject, Resampling

for filename in sPlist:
    
    field = gp.read_file('.../Illisoy/Illinois_shp/county_shape_files/ToJSON/ADAMS.json')
    dst_crs = field.crs
    mydir = '.../Illisoy/WEATHER/Illinois/CLIP/Illi19802022'
    with rio.open(os.path.join(mydir, filename)) as src:
        transform, width, height = calculate_default_transform(
           src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rio.open('.../Illisoy/WEATHER/Illinois/CLIP/Illi19802022/Rej/' + filename, 'w', **kwargs) as dst:
             for i in range(1, src.count + 1):
                reproject(
                   source=rio.band(src, i),
                   destination=rio.band(dst, i),
                   src_transform=src.transform,
                   src_crs=src.crs,
                   dst_transform=transform,
                   dst_crs=dst_crs,
                   resampling=Resampling.nearest)

In [3]:
## clip the county polygons

directory='.../Illisoy/Illinois_shp/county_shape_files/JSON4269'

CSlist = os.listdir(directory)

CSlist

['MASSAC.json',
 'EDWARDS.json',
 'LAKE.json',
 'COLES.json',
 'WARREN.json',
 'GRUNDY.json',
 'MONTGOMERY.json',
 'MCLEAN.json',
 'CLARK.json',
 'CLAY.json',
 'OGLE.json',
 'WINNEBAGO.json',
 'MARSHALL.json',
 'LEE.json',
 'JACKSON.json',
 'BOONE.json',
 'MASON.json',
 'HAMILTON.json',
 'HENRY.json',
 'RANDOLPH.json',
 'ROCK_ISLAND.json',
 'CASS.json',
 'KANKAKEE.json',
 'FRANKLIN.json',
 'UNION.json',
 'MENARD.json',
 'WASHINGTON.json',
 'CARROLL.json',
 'BROWN.json',
 'FAYETTE.json',
 'KNOX.json',
 'TAZEWELL.json',
 'WILLIAMSON.json',
 'DEKALB.json',
 'PEORIA.json',
 'KANE.json',
 'CRAWFORD.json',
 'ST__CLAIR.json',
 'CHAMPAIGN.json',
 'LASALLE.json',
 'FULTON.json',
 'PULASKI.json',
 'MACON.json',
 'LOGAN.json',
 'ADAMS.json',
 'JEFFERSON.json',
 'EFFINGHAM.json',
 'FORD.json',
 'RICHLAND.json',
 'MONROE.json',
 'MCDONOUGH.json',
 'MORGAN.json',
 'VERMILION.json',
 'SCOTT.json',
 'SALINE.json',
 'GALLATIN.json',
 'MOULTRIE.json',
 'CLINTON.json',
 'PERRY.json',
 'WOODFORD.json',
 '

In [6]:
for countyname in CSlist:

    for filename in sPlist:
        
        dirname = '.../Illisoy/WEATHER/Illinois/CLIP/county19802022'
        
        existlist = os.listdir(dirname)
        
        checkitems = Path(countyname).stem + filename
        
        if any(checkitems == element for element in existlist):
            print(f"{checkitems} exists, move to the next file.")
            continue

        mydir = '.../Illisoy/WEATHER/Illinois/CLIP/Illi19802022/Rej'

        with fiona.open(os.path.join(directory, countyname), "r") as shapefile:
            shapes = [feature["geometry"] for feature in shapefile]

        with rio.open(os.path.join(mydir, filename)) as src:
            out_image, out_transform = rio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
 
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})

        with rio.open(os.path.join(dirname, Path(os.path.join(directory, countyname)).stem + filename), "w", **out_meta) as dest:
            dest.write(out_image)

MASSAC1982dayl.nc exists, move to the next file.
MASSAC2015vp.nc exists, move to the next file.
MASSAC1985vp.nc exists, move to the next file.
MASSAC2007tmax.nc exists, move to the next file.
MASSAC1989dayl.nc exists, move to the next file.
MASSAC2014dayl.nc exists, move to the next file.
MASSAC1999dayl.nc exists, move to the next file.
MASSAC1997vp.nc exists, move to the next file.
MASSAC2002tmax.nc exists, move to the next file.
MASSAC1998vp.nc exists, move to the next file.
MASSAC2006prcp.nc exists, move to the next file.
MASSAC1981srad.nc exists, move to the next file.
MASSAC1996dayl.nc exists, move to the next file.
MASSAC2007vp.nc exists, move to the next file.
MASSAC2016dayl.nc exists, move to the next file.
MASSAC1995tmin.nc exists, move to the next file.
MASSAC2003srad.nc exists, move to the next file.
MASSAC1995dayl.nc exists, move to the next file.
MASSAC2015prcp.nc exists, move to the next file.
MASSAC2013vp.nc exists, move to the next file.
MASSAC1989vp.nc exists, move to 

EDWARDS1996tmin.nc exists, move to the next file.
EDWARDS1992tmax.nc exists, move to the next file.
EDWARDS2013prcp.nc exists, move to the next file.
EDWARDS1995srad.nc exists, move to the next file.
EDWARDS1984dayl.nc exists, move to the next file.
EDWARDS1994dayl.nc exists, move to the next file.
EDWARDS2002vp.nc exists, move to the next file.
EDWARDS1987prcp.nc exists, move to the next file.
EDWARDS1983dayl.nc exists, move to the next file.
EDWARDS2000prcp.nc exists, move to the next file.
EDWARDS2002dayl.nc exists, move to the next file.
EDWARDS1983vp.nc exists, move to the next file.
EDWARDS1986prcp.nc exists, move to the next file.
EDWARDS1980tmin.nc exists, move to the next file.
EDWARDS2004tmin.nc exists, move to the next file.
EDWARDS1983prcp.nc exists, move to the next file.
EDWARDS1986vp.nc exists, move to the next file.
EDWARDS1999tmin.nc exists, move to the next file.
EDWARDS1984vp.nc exists, move to the next file.
EDWARDS1988tmax.nc exists, move to the next file.
EDWARDS2

LAKE1985prcp.nc exists, move to the next file.
LAKE1992vp.nc exists, move to the next file.
LAKE2007tmin.nc exists, move to the next file.
LAKE1996tmax.nc exists, move to the next file.
LAKE1997srad.nc exists, move to the next file.
LAKE1993dayl.nc exists, move to the next file.
LAKE1985srad.nc exists, move to the next file.
LAKE1991tmin.nc exists, move to the next file.
LAKE2008vp.nc exists, move to the next file.
LAKE1999srad.nc exists, move to the next file.
LAKE1985tmax.nc exists, move to the next file.
LAKE2007prcp.nc exists, move to the next file.
LAKE1998tmin.nc exists, move to the next file.
LAKE1982tmin.nc exists, move to the next file.
LAKE2009vp.nc exists, move to the next file.
LAKE1987tmin.nc exists, move to the next file.
LAKE2010prcp.nc exists, move to the next file.
LAKE2010srad.nc exists, move to the next file.
LAKE2008tmin.nc exists, move to the next file.
LAKE2014tmax.nc exists, move to the next file.
LAKE2002prcp.nc exists, move to the next file.
LAKE1988vp.nc exist

COLES2009dayl.nc exists, move to the next file.
COLES2006vp.nc exists, move to the next file.
COLES2007dayl.nc exists, move to the next file.
COLES2008dayl.nc exists, move to the next file.
COLES1997prcp.nc exists, move to the next file.
COLES1987tmax.nc exists, move to the next file.
COLES2011tmin.nc exists, move to the next file.
COLES1998tmax.nc exists, move to the next file.
COLES2015dayl.nc exists, move to the next file.
COLES1983tmax.nc exists, move to the next file.
COLES2012tmax.nc exists, move to the next file.
COLES2013tmax.nc exists, move to the next file.
COLES2004srad.nc exists, move to the next file.
COLES2011srad.nc exists, move to the next file.
COLES2006tmax.nc exists, move to the next file.
COLES2001dayl.nc exists, move to the next file.
COLES2016vp.nc exists, move to the next file.
COLES1994prcp.nc exists, move to the next file.
COLES1991srad.nc exists, move to the next file.
COLES1986tmin.nc exists, move to the next file.
COLES2011dayl.nc exists, move to the next fi

WARREN2012prcp.nc exists, move to the next file.
WARREN2001vp.nc exists, move to the next file.
WARREN2012tmin.nc exists, move to the next file.
WARREN1990tmin.nc exists, move to the next file.
WARREN2003dayl.nc exists, move to the next file.
WARREN2006tmin.nc exists, move to the next file.
WARREN1991prcp.nc exists, move to the next file.
WARREN2014prcp.nc exists, move to the next file.
WARREN2008srad.nc exists, move to the next file.
GRUNDY1982dayl.nc exists, move to the next file.
GRUNDY2015vp.nc exists, move to the next file.
GRUNDY1985vp.nc exists, move to the next file.
GRUNDY2007tmax.nc exists, move to the next file.
GRUNDY1989dayl.nc exists, move to the next file.
GRUNDY2014dayl.nc exists, move to the next file.
GRUNDY1999dayl.nc exists, move to the next file.
GRUNDY1997vp.nc exists, move to the next file.
GRUNDY2002tmax.nc exists, move to the next file.
GRUNDY1998vp.nc exists, move to the next file.
GRUNDY2006prcp.nc exists, move to the next file.
GRUNDY1981srad.nc exists, move

...

TAZEWELL1981tmin.nc exists, move to the next file.
TAZEWELL2001prcp.nc exists, move to the next file.
TAZEWELL2017dayl.nc exists, move to the next file.
TAZEWELL2017srad.nc exists, move to the next file.
TAZEWELL1982srad.nc exists, move to the next file.
TAZEWELL1995prcp.nc exists, move to the next file.
TAZEWELL2002srad.nc exists, move to the next file.
TAZEWELL2000dayl.nc exists, move to the next file.
TAZEWELL2009srad.nc exists, move to the next file.
TAZEWELL1988prcp.nc exists, move to the next file.
TAZEWELL1993srad.nc exists, move to the next file.
TAZEWELL1992dayl.nc exists, move to the next file.
TAZEWELL2007srad.nc exists, move to the next file.
TAZEWELL1988dayl.nc exists, move to the next file.
TAZEWELL1990prcp.nc exists, move to the next file.
TAZEWELL2017prcp.nc exists, move to the next file.
TAZEWELL2005srad.nc exists, move to the next file.
TAZEWELL1993vp.nc exists, move to the next file.
TAZEWELL1981prcp.nc exists, move to the next file.
TAZEWELL1998prcp.nc exists, move 

WILLIAMSON2004prcp.nc exists, move to the next file.
WILLIAMSON1983srad.nc exists, move to the next file.
WILLIAMSON2008tmax.nc exists, move to the next file.
WILLIAMSON2000srad.nc exists, move to the next file.
WILLIAMSON1991tmax.nc exists, move to the next file.
WILLIAMSON1992srad.nc exists, move to the next file.
WILLIAMSON1994tmax.nc exists, move to the next file.
WILLIAMSON2005vp.nc exists, move to the next file.
WILLIAMSON2016tmin.nc exists, move to the next file.
WILLIAMSON1984tmax.nc exists, move to the next file.
WILLIAMSON1981vp.nc exists, move to the next file.
WILLIAMSON2013srad.nc exists, move to the next file.
WILLIAMSON2014srad.nc exists, move to the next file.
WILLIAMSON2012prcp.nc exists, move to the next file.
WILLIAMSON2001vp.nc exists, move to the next file.
WILLIAMSON2012tmin.nc exists, move to the next file.
WILLIAMSON1990tmin.nc exists, move to the next file.
WILLIAMSON2003dayl.nc exists, move to the next file.
WILLIAMSON2006tmin.nc exists, move to the next file.

KANE1990tmax.nc exists, move to the next file.
KANE1980vp.nc exists, move to the next file.
KANE2017vp.nc exists, move to the next file.
KANE2014vp.nc exists, move to the next file.
KANE1995vp.nc exists, move to the next file.
KANE2015srad.nc exists, move to the next file.
KANE1995tmax.nc exists, move to the next file.
KANE1989tmax.nc exists, move to the next file.
KANE2016tmax.nc exists, move to the next file.
KANE2001tmax.nc exists, move to the next file.
KANE1980srad.nc exists, move to the next file.
KANE1997tmin.nc exists, move to the next file.
KANE1987dayl.nc exists, move to the next file.
KANE1996tmin.nc exists, move to the next file.
KANE1992tmax.nc exists, move to the next file.
KANE2013prcp.nc exists, move to the next file.
KANE1995srad.nc exists, move to the next file.
KANE1984dayl.nc exists, move to the next file.
KANE1994dayl.nc exists, move to the next file.
KANE2002vp.nc exists, move to the next file.
KANE1987prcp.nc exists, move to the next file.
KANE1983dayl.nc exists,

CRAWFORD1996tmax.nc exists, move to the next file.
CRAWFORD1997srad.nc exists, move to the next file.
CRAWFORD1993dayl.nc exists, move to the next file.
CRAWFORD1985srad.nc exists, move to the next file.
CRAWFORD1991tmin.nc exists, move to the next file.
CRAWFORD2008vp.nc exists, move to the next file.
CRAWFORD1999srad.nc exists, move to the next file.
CRAWFORD1985tmax.nc exists, move to the next file.
CRAWFORD2007prcp.nc exists, move to the next file.
CRAWFORD1998tmin.nc exists, move to the next file.
CRAWFORD1982tmin.nc exists, move to the next file.
CRAWFORD2009vp.nc exists, move to the next file.
CRAWFORD1987tmin.nc exists, move to the next file.
CRAWFORD2010prcp.nc exists, move to the next file.
CRAWFORD2010srad.nc exists, move to the next file.
CRAWFORD2008tmin.nc exists, move to the next file.
CRAWFORD2014tmax.nc exists, move to the next file.
CRAWFORD2002prcp.nc exists, move to the next file.
CRAWFORD1988vp.nc exists, move to the next file.
CRAWFORD1988srad.nc exists, move to t

CHAMPAIGN1990tmax.nc exists, move to the next file.
CHAMPAIGN1980vp.nc exists, move to the next file.
CHAMPAIGN2017vp.nc exists, move to the next file.
CHAMPAIGN2014vp.nc exists, move to the next file.
CHAMPAIGN1995vp.nc exists, move to the next file.
CHAMPAIGN2015srad.nc exists, move to the next file.
CHAMPAIGN1995tmax.nc exists, move to the next file.
CHAMPAIGN1989tmax.nc exists, move to the next file.
CHAMPAIGN2016tmax.nc exists, move to the next file.
CHAMPAIGN2001tmax.nc exists, move to the next file.
CHAMPAIGN1980srad.nc exists, move to the next file.
CHAMPAIGN1997tmin.nc exists, move to the next file.
CHAMPAIGN1987dayl.nc exists, move to the next file.
CHAMPAIGN1996tmin.nc exists, move to the next file.
CHAMPAIGN1992tmax.nc exists, move to the next file.
CHAMPAIGN2013prcp.nc exists, move to the next file.
CHAMPAIGN1995srad.nc exists, move to the next file.
CHAMPAIGN1984dayl.nc exists, move to the next file.
CHAMPAIGN1994dayl.nc exists, move to the next file.
CHAMPAIGN2002vp.nc e

In [ ]:
################ start calculating daily observations from HERE!!! ##################

In [8]:
for year in [str(x) for x in list(range(1980, 2023))]:

  # find all the weather rasters in the year

    dirname = '.../Illisoy/WEATHER/Illinois/CLIP/county19802022'
    
    suffix = '.csv'

    files = glob.glob(dirname + '**/*' + year + '*' + '.nc', recursive=False)

    files = [f for f in files if os.path.isfile(f)]

    ## list of county names

    directory='.../Illisoy/Illinois_shp/county_shape_files/JSON4269'

    CSlist = os.listdir(directory)
    
    SW = "%s-W%s" % (year, 20)
    EW = "%s-W%s" % (year, 45)

    import datetime 

    ## NASS week is ISO week
    StartDate = datetime.datetime.strptime(SW + '-1', "%G-W%V-%u")
    EndDate = datetime.datetime.strptime(EW + '-1', "%G-W%V-%u") + datetime.timedelta(days=6)
    SDOY = StartDate.timetuple().tm_yday
    EDOY = EndDate.timetuple().tm_yday

    itemsarray = ['County', 'Weather'] 

    itemsarray.extend(range(SDOY, EDOY+1))

    itemdict = {n: n for n in itemsarray}  

    allweather = []

    for countyname in CSlist:
    
        counwealist = []

        counwealist += [each for each in files if Path(countyname).stem == Path(each).stem.split(year)[0]]

        for weatherfile in counwealist:

            raster = rio.open(weatherfile)
    
            sbv = [0 for x in range(raster.count)]

            for band in range(1, raster.count+1): #The +1 allows the writing of the last band
                single_band = raster.read(band)
                single_band[single_band==-9999]=np.nan
                sbv[band-1] = np.nanmean(single_band)
        
                sbvs = sbv[SDOY-1 : EDOY]
    
                def substring_after(s, delim):
                  return s.partition(delim)[2]
    
                countyweatherarray = [Path(countyname).stem, substring_after(Path(weatherfile).stem, year)] 
    
                countyweatherarray.extend(sbvs)
        
                withkey_dict = dict(zip(itemsarray, countyweatherarray))
    
            allweather.append(withkey_dict)      

    with open(os.path.join(year + 'DailyCounWea' + suffix), 'w') as f:  
        w = csv.DictWriter(f, itemdict.keys())
        w.writeheader()
        w.writerows(allweather)